##Pre-procesamiento de Datos de Vuelos en Tiempo Real

# Resumen

Se realizó un proceso de ETL en tiempo real implementado en Databricks para procesar datos de tráfico aéreo. El objetivo es ingerir datos crudos de vuelos desde una fuente de streaming (Kafka), aplicar una serie de transformaciones para limpiar, enriquecer y estructurar la información, y finalmente, almacenar los datos procesados en una tabla optimizada (Delta Lake) para su posterior análisis y visualización en Power BI.

# Diagrama flujo de datos

Colocar imagen

El pipeline se divide en tres fases principales: Ingesta, Transformación y Carga.


**Fase 1: Ingesta desde Kafka (Extract)**

El primer paso es conectarse al flujo de datos en vivo.




*   Fuente: Los datos se leen desde el tópico de Kafka flight_stream, el cual recibe la información de la API de OpenSky Network.
*   Motor: Se utiliza spark.readStream, el componente de Spark para leer flujos de datos continuos.
*   Configuración:
1.   kafka.bootstrap.servers: Especifica la dirección del servidor de Kafka para establecer la conexión.

2.   subscribe: Indica a Spark que se suscriba al tópico flight_stream.

3.   startingOffsets: "latest": Configura el stream para que procese solo los datos nuevos que llegan desde el momento en que se inicia el pipeline, ignorando antiguos.

**Fase 2: Parseo, Transformación y Enriquecimiento (Transform)**

Esta es la fase del pipeline donde los datos crudos se convierten en información de valor.

**2.1. Parseo y Estructuración del JSON**

Los mensajes de Kafka llegan como un value binario. El primer paso es darles una estructura útil.

1.   Definición de Esquema: Se define un schema explícito con StructType asignando un tipo de dato correcto a cada campo (StringType, LongType, DoubleType, etc.).
2.   Aplicación del Esquema: La función from_json se utiliza para parsear el string del mensaje y aplicar el esquema definido, convirtiendo un string plano en una estructura de columnas anidada. Con select("data.*") se "aplanan" estas columnas para que sean accesibles directamente.

**2.2. Transformaciones y Creación de Nuevas Columnas**

Una vez con los datos estructurados, se aplican una serie de transformaciones usando withColumn para limpiar y enriquecer el dataset.



1.   Conversión de Timestamps
*   FechaHora_Posicion: El campo time_position (un timestamp UNIX en segundos) se convierte a un formato de fecha y hora legible (TimestampType) usando from_unixtime.
*   FechaHora_Ingesta: El campo timestamp_ingest (un string en formato ISO 8601) se convierte a TimestampType usando to_timestamp.
   
2.   Conversión de Unidades
*   Altitud_Pies: Se convierte la altitud barométrica de metros a pies (baro_altitude * 3.28084).
*   Velocidad_Kmh: Se convierte la velocidad de metros/segundo a kilómetros/hora (velocity * 3.6).

3.   Atributos Categóricos
*   estado_de_vuelo: Se crea una columna categórica que clasifica el estado del vuelo basándose en si está en tierra, su velocidad y su altitud.
*   Indicador_Congestion: Se crea un indicador booleano que identifica vuelos en un patrón de posible congestión (cierto rango de velocidad y altitud).
*   Cuadrante: Se clasifica cada vuelo en un cuadrante geográfico predefinido basado en su latitud y longitud.

4.   Desglose de Fechas
*   Se extraen los componentes de día, mes y hora de las fechas de posición e ingesta.

**Fase 3: Carga en Delta Lake (Load)**

El último paso es escribir el DataFrame transformado en una tabla final que sea robusta.

*   Destino: Se utiliza el formato Delta Lake.
*   Modo de Salida: Se usa outputMode("append"), lo que significa que cada nuevo registro procesado se añade a la tabla Delta sin modificar los datos existentes.
*   Creación de la Tabla SQL: El comando %sql CREATE TABLE registra la ubicación de los archivos Delta como una tabla SQL (flights_gold). Esto hace que los datos sean fácilmente accesibles para ser consultados con SQL estándar.